In [35]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?
import sys

from siuba import *
import pandas as pd
import geopandas as gpd
import datetime as dt

from rt_analysis import rt_parser
from rt_analysis import rt_filter_map_plot

import shared_utils

In [36]:
import datetime as dt
import os
import re
import time
from pathlib import Path
from typing import Union

import branca
import dask_geopandas as dg
import folium
import gcsfs
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import siuba  # need for type hints
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.tables import tbls
from numba import jit
from shared_utils import geography_utils, gtfs_utils, gtfs_utils_v2, map_utils, utils
from siuba import *

# Migrate existing rt_analysis to use the v2 warehouse 

* mostly just changing queries around

In [41]:
# date for v2 testing
analysis_date = dt.date(2023, 3, 15)

## v2 organization/datasets/feeds

In [46]:
daily_service = (tbls.mart_gtfs.fct_daily_feed_scheduled_service_summary()
    >> select(_.schedule_gtfs_dataset_key == _.gtfs_dataset_key,
             _.feed_key, _.activity_date)
                )

org_feeds_datasets = (tbls.mart_transit_database.dim_provider_gtfs_data()
    >> filter(_._is_current, _.reports_site_assessed,
            _.organization_name == 'Alameda-Contra Costa Transit District')
            ## think more about how to start/persist org level identifiers...
            ## could be an attribute, or in any case leave first index table as sql...
    >> inner_join(_, daily_service, by = 'schedule_gtfs_dataset_key')
    >> filter(_.activity_date == analysis_date)
    # >> distinct(_.feed_key)
    )

In [47]:
org_feeds_datasets

,key,guidelines_assessed,reports_site_assessed,organization_key,organization_name,organization_itp_id,organization_hubspot_company_record_id,organization_ntd_id,organization_source_record_id,service_key,...,trip_updates_source_record_id,schedule_gtfs_dataset_key,service_alerts_gtfs_dataset_key,vehicle_positions_gtfs_dataset_key,trip_updates_gtfs_dataset_key,_valid_from,_valid_to,_is_current,activity_date,feed_key
0,0f2aa25ebedc36f379a1b8efdc67cced,False,True,c73f84833af54e86518de137ae67ef3f,Alameda-Contra Costa Transit District,4,1880690867,90014,recOZgevYf7Jimm9L,d1444c8a6478da002d65324246682885,...,recFOyTNrrrVpozMy,444700afe086ed24e3cb888cecd3037c,None,4bb240cf480589e58c6e58d06d6fb72c,6aba5c314d297bcedd7ec86fcaf9361c,2023-03-14 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,True,2023-03-15,84eb243b2456b2d4a0b57d4eefe31000
1,3a312132133aa386ca04b2cc9a9e9b7e,False,True,c73f84833af54e86518de137ae67ef3f,Alameda-Contra Costa Transit District,4,1880690867,90014,recOZgevYf7Jimm9L,b0aad71e4c04e4c647b5d36b07e5cb63,...,recFOyTNrrrVpozMy,444700afe086ed24e3cb888cecd3037c,None,4bb240cf480589e58c6e58d06d6fb72c,6aba5c314d297bcedd7ec86fcaf9361c,2023-03-14 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,True,2023-03-15,84eb243b2456b2d4a0b57d4eefe31000
2,2412ea392ec31f99e10f6879f6ba8a26,False,True,c73f84833af54e86518de137ae67ef3f,Alameda-Contra Costa Transit District,4,1880690867,90014,recOZgevYf7Jimm9L,26963c5207c102bbdca4777e580765fd,...,recFOyTNrrrVpozMy,444700afe086ed24e3cb888cecd3037c,None,4bb240cf480589e58c6e58d06d6fb72c,6aba5c314d297bcedd7ec86fcaf9361c,2023-03-14 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,True,2023-03-15,84eb243b2456b2d4a0b57d4eefe31000
3,0f649b4c8db460fb41b490967aba9bb0,False,True,c73f84833af54e86518de137ae67ef3f,Alameda-Contra Costa Transit District,4,1880690867,90014,recOZgevYf7Jimm9L,4b74478a08534933b97246e56f065742,...,recCsrEnH5Bjgmbfm,d7dcb48dbed269f345db12d1884a9e33,None,6d1991ecbdee1e06031cda8761757f03,5c3e65766dda65958cf4da845286c0d5,2023-03-14 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,True,2023-03-15,aa047b2e150d2833f6713867b2c4ae71


In [38]:
fs = gcsfs.GCSFileSystem()
BUCKET_NAME = "calitp-analytics-data"
VP_FILE_PATH = f"gs://{BUCKET_NAME}/data-analyses/rt_segment_speeds/"

# set system time
os.environ["TZ"] = "America/Los_Angeles"
time.tzset()

In [28]:
ac_v1 = shared_utils.rt_utils.get_vehicle_positions(4, dt.date(2022, 10, 17))

found parquet


In [14]:
ac_v1 >> head(3)

,calitp_itp_id,calitp_url_number,vehicle_timestamp,entity_id,vehicle_id,trip_id,vehicle_longitude,vehicle_latitude
0,4,0,2022-10-17 00:59:37,None,1361,783020,-122.27368,37.805170
1,4,0,2022-10-17 00:59:42,None,2251,14070010,-122.15236,37.743004
2,4,0,2022-10-17 00:59:40,None,2245,13273010,-122.29322,37.838596


In [23]:
# https://github.com/cal-itp/data-analyses/blob/main/open_data/download_vehicle_positions.py
# design these tools to read this, filter to organization, write out...
# starts with warehouse vehicle locations table
vp_all = gpd.read_parquet(f'{VP_FILE_PATH}vp_2023-03-15.parquet')

In [48]:
org_feeds_df = org_feeds_datasets >> collect()

In [49]:
org_feeds_df.columns

Index(['key', 'guidelines_assessed', 'reports_site_assessed',
       'organization_key', 'organization_name', 'organization_itp_id',
       'organization_hubspot_company_record_id', 'organization_ntd_id',
       'organization_source_record_id', 'service_key', 'service_name',
       'service_source_record_id', 'gtfs_service_data_customer_facing',
       'regional_feed_type', 'associated_schedule_gtfs_dataset_key',
       'schedule_gtfs_dataset_name', 'schedule_source_record_id',
       'service_alerts_gtfs_dataset_name', 'service_alerts_source_record_id',
       'vehicle_positions_gtfs_dataset_name',
       'vehicle_positions_source_record_id', 'trip_updates_gtfs_dataset_name',
       'trip_updates_source_record_id', 'schedule_gtfs_dataset_key',
       'service_alerts_gtfs_dataset_key', 'vehicle_positions_gtfs_dataset_key',
       'trip_updates_gtfs_dataset_key', '_valid_from', '_valid_to',
       '_is_current', 'activity_date', 'feed_key'],
      dtype='object')

In [51]:
org_vp = vp_all >> filter(_.gtfs_dataset_key.isin(org_feeds_df.vehicle_positions_gtfs_dataset_key))

In [55]:
# success, includes AC and DBX as expected!
(org_vp >> select(_.trip_id, _.geometry)).sample(1000).explore()

In [25]:
org_vl.columns

Index(['gtfs_dataset_key', 'vehicle_timestamp', 'vehicle_id', 'trip_id',
       'position_longitude', 'position_latitude'],
      dtype='object')

In [27]:
## (gdf >> select(_.geometry, _.trip_id) >> head(1000)).explore()

In [28]:
import sys

In [29]:
new_size = sys.getsizeof(org_vl)
new_size

380786665

In [30]:
old_size = sys.getsizeof(ac_v1)
old_size

345707533

In [31]:
## OK after selecting similar subset of columns :) 
new_size / old_size

1.1014705456244716

## New Trips Query

In [ ]:
old_trips = shared_utils.rt_utils.get_trips(4, dt.date(2022, 10, 17))

In [ ]:
old_trips >> head(3)

In [57]:
org_trips = (tbls.mart_gtfs.fct_daily_scheduled_trips()
             >> filter(_.activity_date == analysis_date)
             >> inner_join(_, org_feeds_datasets >> select(_.schedule_gtfs_dataset_key),
                           on = {'gtfs_dataset_key':'schedule_gtfs_dataset_key'})
             >> select(_.trip_key, _.gtfs_dataset_key, _.activity_date,
                       _.trip_id, _.route_id, _.route_short_name,
                       _.shape_id, _.direction_id, _.route_type,
                       _.route_long_name, _.route_desc
                      )
             # no longer need to join in routes, thanks v2 warehouse!
             >> collect()
            )

In [58]:
org_trips >> head(3)

,trip_key,gtfs_dataset_key,activity_date,trip_id,route_id,route_short_name,shape_id,direction_id,route_type,route_long_name,route_desc
0,6009197e215df5f49ecc8348967c4b20,444700afe086ed24e3cb888cecd3037c,2023-03-15,6782020,O,O,shp-O-03,1,3,Santa Clara - Encinal Transbay,None
1,6009197e215df5f49ecc8348967c4b20,444700afe086ed24e3cb888cecd3037c,2023-03-15,6782020,O,O,shp-O-03,1,3,Santa Clara - Encinal Transbay,None
2,6009197e215df5f49ecc8348967c4b20,444700afe086ed24e3cb888cecd3037c,2023-03-15,6782020,O,O,shp-O-03,1,3,Santa Clara - Encinal Transbay,None


In [ ]:
new_size = sys.getsizeof(org_trips)
new_size

In [ ]:
old_size = sys.getsizeof(old_trips)
old_size

In [ ]:
## OK after selecting similar subset of columns :) 
new_size / old_size

## New ST Query

In [22]:
old_st = shared_utils.rt_utils.get_stop_times(4, dt.date(2022, 10, 17))

found parquet


In [23]:
old_st >> head(3)

,calitp_itp_id,trip_id,stop_id,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,...,stop_time_key,calitp_deleted_at,stop_sequence,stop_time_continuous_pickup,stop_time_continuous_drop_off,stop_sequence_rank,arrival_ts,departure_ts,trip_key,departure_hour
0,4,12731020,53832,7:51:34,7:51:34,33 PIEDMONT TO HIGHLAND WAY VIA OAKLAND AVE,0,0,15292.32000000,0,...,6674480576887928016,2022-12-04,57,None,None,57,28294,28294,4622753192079563450,7
1,4,2699020,51832,12:17:36,12:17:36,72M JACK LONDON SQ VIA SAN PABLO AVE DOWNTOWN ...,0,0,18080.36000000,0,...,7067786241977062485,2022-12-04,60,None,None,60,44256,44256,491570519637676407,12
2,4,3198020,3099,18:21:08,18:21:08,57 FOOTHILL SQUARE VIA MACARTHUR BLVD,0,0,18275.13,0,...,4813443666059748000,2022-12-04,57,None,None,57,66068,66068,-6354043175372643119,18


In [9]:
analysis_date

datetime.date(2023, 2, 17)

In [59]:
## still required to filter to trips running on date...
trips_day_filtered = (tbls.mart_gtfs.fct_daily_scheduled_trips()
                      >> filter(_.activity_date == analysis_date)
                     )
org_st = (
    (org_feeds_datasets >> distinct(_.feed_key))
    >> inner_join(_, tbls.mart_gtfs.dim_stop_times(), on = 'feed_key')
    >> inner_join(_, trips_day_filtered, on = ['feed_key', 'trip_id'])
    >> inner_join(_, (tbls.mart_gtfs.dim_stops() >> select(_.feed_key,
                        _.stop_id, _.stop_name))
                  , on = ['feed_key', 'stop_id'])
    >> select(_.feed_key, _.gtfs_dataset_key, _.trip_id,
             _.stop_id, _.arrival_time, _.departure_time,
             _.timepoint, _.stop_sequence, _.continuous_drop_off,
             _.continuous_pickup)
    )

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'


In [60]:
org_st = org_st >> collect()

In [61]:
old_st.columns

NameError: name 'old_st' is not defined

In [ ]:
org_st.columns

In [53]:
new_size = sys.getsizeof(org_st)
new_size

120812527

In [54]:
old_size = sys.getsizeof(old_st)
old_size

206354241

In [55]:
## hey look this one's smaller :) 
new_size / old_size

0.585461807882107

## New Shapes Query

In [56]:
old_shp = shared_utils.rt_utils.get_routelines(4, dt.date(2022, 10, 17))

In [57]:
old_shp >> head(3)

,calitp_itp_id,calitp_url_number,shape_id,geometry
0,4,0,shp-95-53,"LINESTRING (-180326.410 -34738.218, -180345.10..."
1,4,0,shp-65-12,"LINESTRING (-199094.894 -13747.291, -199076.60..."
2,4,0,shp-86-52,"LINESTRING (-183767.650 -36536.552, -183750.70..."


In [80]:
org_shp = (
    org_feed_keys
    >> inner_join(_, tbls.mart_gtfs.dim_shapes_arrays(), on = 'feed_key')
    >> inner_join(_, (trips_day_filtered >> distinct(
                        _.feed_key, _.shape_id)),
                  on = ['feed_key', 'shape_id'])
    >> select(_.feed_key, _.shape_id, _.pt_array)
    )

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'


In [81]:
org_shp = org_shp >> collect()

In [82]:
def linestring_from_wkt(wkt_list):
    '''
    Use shapely to create linestring geometries from wkt points arrays
    in tables such as dim_shapes_arrays
    
    wkt_list: list of points in wkt string format
    '''
    ## shapely 1.85; 2.0 has new top-level from_wkt...
    pt_list = [shapely.wkt.loads(pt) for pt in wkt_list]
    linestring = shapely.geometry.LineString(pt_list)
    return linestring

In [83]:
org_shp['geometry'] = org_shp.pt_array.apply(linestring_from_wkt)

In [84]:
org_shp = org_shp >> select(-_.pt_array)

In [85]:
org_shp

,feed_key,shape_id,geometry
0,2e0478675d2b2bcd6c93da9354d92755,shp-239-59,"LINESTRING (-121.97549 37.556958, -121.9753 37..."
1,2e0478675d2b2bcd6c93da9354d92755,shp-65-14,"LINESTRING (-122.246738 37.880352, -122.246696..."
2,2e0478675d2b2bcd6c93da9354d92755,shp-200-16,"LINESTRING (-121.976025 37.557496, -121.975895..."
3,2e0478675d2b2bcd6c93da9354d92755,shp-33-17,"LINESTRING (-122.209505 37.825505, -122.209488..."
4,2e0478675d2b2bcd6c93da9354d92755,shp-72-33,"LINESTRING (-122.27785 37.796567, -122.277628 ..."
...,...,...,...
323,aa047b2e150d2833f6713867b2c4ae71,DB10036,"LINESTRING (-122.1502733 37.395688, -122.15039..."
324,aa047b2e150d2833f6713867b2c4ae71,DB10039,"LINESTRING (-122.0172182 37.5902461, -122.0170..."
325,aa047b2e150d2833f6713867b2c4ae71,DB10045,"LINESTRING (-122.1499304 37.3954966, -122.1496..."
326,aa047b2e150d2833f6713867b2c4ae71,DB0084,"LINESTRING (-122.017422 37.5903968, -122.01741..."
